
# Aprendizaje por Refuerzo (Reinforcement Learning)

El **Aprendizaje por Refuerzo (RL)** es una rama del aprendizaje automático en la que un agente aprende a tomar decisiones óptimas mediante la interacción con un entorno, maximizando una señal de recompensa.

---

## Fundamentos del RL

- **Agente**: quien toma decisiones.
- **Entorno (Environment)**: donde actúa el agente.
- **Estado (State)**: situación actual del entorno.
- **Acción (Action)**: decisión del agente.
- **Recompensa (Reward)**: señal que recibe el agente tras actuar.
- **Política (Policy)**: estrategia de acción del agente.
- **Función de valor (Value Function)**: estima la utilidad futura esperada.

---

## Ecuación de Bellman (Valor Esperado):

\[
V(s) = \mathbb{E}[R_{t+1} + \gamma V(s') | s]
\]

Donde:
- \( \gamma \): factor de descuento.
- \( s' \): nuevo estado.
- \( R_{t+1} \): recompensa inmediata.




## Estado del Arte en RL

Hoy en día, el aprendizaje por refuerzo ha sido exitosamente aplicado en múltiples áreas:

- **Juegos**: AlphaGo, Dota 2, StarCraft II
- **Robótica**: control de brazos robóticos
- **Finanzas**: estrategias de inversión
- **Vehículos autónomos**: planificación y navegación
- **Sistemas de recomendación**: decisiones dinámicas

---

### Algoritmos comunes

| Categoría         | Algoritmos                  |
|-------------------|-----------------------------|
| Basados en valores| Q-Learning, SARSA           |
| Basados en políticas | REINFORCE, PPO, A3C        |
| Actor-Critic      | DDPG, TD3, A3C              |


In [ ]:

import numpy as np
import matplotlib.pyplot as plt



## Ejemplo 1: Navegación en una cuadrícula (Grid World)

El agente se mueve en una cuadrícula con obstáculos y aprende a llegar al objetivo con la mayor recompensa posible.


In [ ]:

# Definición simple de entorno tipo grid
grid = np.array([
    [-1, -1, -1, +10],
    [-1, None, -1, -10],
    [-1, -1, -1, -1]
])

# Posiciones válidas
states = [(i, j) for i in range(3) for j in range(4) if grid[i, j] is not None]

actions = {'up': (-1, 0), 'down': (1, 0), 'left': (0, -1), 'right': (0, 1)}

def move(state, action):
    i, j = state
    di, dj = actions[action]
    new_state = (i + di, j + dj)
    if new_state in states:
        return new_state
    return state

# Inicializar Q
Q = {(s, a): 0 for s in states for a in actions}

# Q-learning simple
alpha, gamma, epsilon = 0.1, 0.9, 0.1
for episode in range(500):
    state = (2, 0)
    while state not in [(0, 3), (1, 3)]:
        if np.random.rand() < epsilon:
            action = np.random.choice(list(actions))
        else:
            action = max(actions, key=lambda a: Q[state, a])
        next_state = move(state, action)
        reward = grid[next_state]
        best_next = max(Q[next_state, a] for a in actions)
        Q[state, action] += alpha * (reward + gamma * best_next - Q[state, action])
        state = next_state

# Mostrar valores aprendidos
for s in sorted(states):
    best = max(actions, key=lambda a: Q[s, a])
    print(f"Mejor acción en {s}: {best}")



## Ejemplo 2: Aprendizaje con OpenAI Gym (CartPole)

El agente aprende a mantener una barra en equilibrio sobre un carro.


In [ ]:

import gym

env = gym.make("CartPole-v1", render_mode="rgb_array")
obs = env.reset()[0]

for _ in range(50):
    action = env.action_space.sample()
    obs, reward, done, _, _ = env.step(action)
    if done:
        obs = env.reset()[0]
env.close()
print("Ejemplo de interacción aleatoria completado.")



## Ejemplo 3: Multi-Armed Bandit

Simulación de toma de decisiones con varias opciones (máquinas tragamonedas).


In [ ]:

# 3 máquinas con distintas probabilidades
probs = [0.2, 0.5, 0.8]
n = 1000
counts = [0] * 3
values = [0.0] * 3

def pull(arm):
    return np.random.rand() < probs[arm]

for t in range(1, n + 1):
    epsilon = 0.1
    if np.random.rand() < epsilon:
        arm = np.random.choice(3)
    else:
        arm = np.argmax(values)
    reward = pull(arm)
    counts[arm] += 1
    values[arm] += (reward - values[arm]) / counts[arm]

plt.bar(range(3), values)
plt.title("Valor estimado por máquina")
plt.show()



## Ejemplo 4: Control de temperatura inteligente

Simulación de RL para encender/apagar calefacción en función del confort y ahorro energético.


In [ ]:

# Estados: temperaturas
temps = list(range(15, 26))  # 15°C a 25°C
actions = ['ON', 'OFF']
Q = {(t, a): 0 for t in temps for a in actions}

for _ in range(1000):
    t = np.random.choice(temps)
    a = np.random.choice(actions)
    reward = -abs(22 - t) if a == 'OFF' else -1  # confort o costo energético
    best = max(Q[t, a2] for a2 in actions)
    Q[t, a] += 0.1 * (reward + 0.9 * best - Q[t, a])

for t in temps:
    best_a = max(actions, key=lambda a: Q[t, a])
    print(f"A {t}°C => Acción óptima: {best_a}")



## Ejemplo 5: Estrategia de compra/venta (RL simplificado)

El agente aprende cuándo comprar, vender o mantener una acción en función de una señal de mercado.


In [ ]:

prices = np.sin(np.linspace(0, 20, 100)) + np.random.normal(0, 0.1, 100)
actions = ['buy', 'sell', 'hold']
Q = {(i, a): 0 for i in range(len(prices)) for a in actions}
inventory = []

for _ in range(100):
    for i in range(len(prices) - 1):
        a = np.random.choice(actions)
        reward = 0
        if a == 'buy':
            inventory.append(prices[i])
        elif a == 'sell' and inventory:
            reward = prices[i] - inventory.pop(0)
        Q[i, a] += 0.01 * (reward - Q[i, a])

# Mostrar mejores acciones aprendidas
for i in range(10):
    print(f"T={i}, Mejor acción:", max(actions, key=lambda a: Q[i, a]))



## Aprendizaje por Refuerzo Profundo (Deep Reinforcement Learning)

El **Deep Reinforcement Learning (DRL)** combina el aprendizaje por refuerzo con redes neuronales profundas, permitiendo al agente aprender políticas directamente desde observaciones complejas como imágenes, texto o vectores continuos de alta dimensión.

---

### Librería recomendada: `stable-baselines3`

```bash
pip install stable-baselines3[extra]
```

Modelos disponibles:
- `DQN` (Deep Q-Network)
- `PPO` (Proximal Policy Optimization)
- `A2C` (Advantage Actor-Critic)



### Ejemplo: PPO con `CartPole-v1` (estable y robusto)

Entrenaremos un agente con `stable-baselines3` usando PPO.


In [ ]:

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env

# Crear entorno vectorizado
env = make_vec_env("CartPole-v1", n_envs=1)

# Entrenar modelo
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=5000)
print("Entrenamiento PPO completo.")


In [ ]:

obs = env.reset()
for _ in range(100):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = env.step(action)
print("Evaluación completada.")
